# <span style="color:blue">[</span> lab-12-2-char-seq-rnn <span style="color:blue">]</span>
<p>출처: <a href="http://hunkim.github.io/ml/" title="모두를 위한 머신러닝과 딥러닝의 강의" target="blank">모두를 위한 머신러닝과 딥러닝의 강의</a></p> <br/>
> [Tensorflow Document(Tensor Transformations)](https://www.tensorflow.org/api_guides/python/array_ops)  <br/>
> [CS 20SI: Tensorflow for Deep Learning Research](http://web.stanford.edu/class/cs20si/)

In [1]:
# Lab 12 Character Sequence RNN
import os
# silence INFO logs set it to 1
# filter out WARNING set it to 2 
# silence ERROR logs (not recommended) set it to 3
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import numpy as np

In [2]:
tf.set_random_seed(777)  # reproducibility

sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ell

In [3]:
X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [4]:
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

In [5]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(outputs, num_classes, activation_fn=None)

In [6]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [7]:
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))


'''
0 loss: 2.35377 Prediction: uuuuuuuuuuuuuuu
1 loss: 2.21383 Prediction: yy you y    you
2 loss: 2.04317 Prediction: yy yoo       ou
3 loss: 1.85869 Prediction: yy  ou      uou
4 loss: 1.65096 Prediction: yy you  a   you
5 loss: 1.40243 Prediction: yy you yan  you
6 loss: 1.12986 Prediction: yy you wann you
7 loss: 0.907699 Prediction: yy you want you
8 loss: 0.687401 Prediction: yf you want you
9 loss: 0.508868 Prediction: yf you want you
10 loss: 0.379423 Prediction: yf you want you
11 loss: 0.282956 Prediction: if you want you
12 loss: 0.208561 Prediction: if you want you

...

'''

0 loss: 2.31902 Prediction: uuuuouuuuouuuou
1 loss: 2.19079 Prediction: u uuouuuooooooo
2 loss: 2.05857 Prediction: y   ou  uo  uou
3 loss: 1.87971 Prediction: y   ou want uou
4 loss: 1.6545 Prediction: y   ou want oou
5 loss: 1.34818 Prediction: y   ou want you
6 loss: 1.05922 Prediction: y   ou want you
7 loss: 0.840225 Prediction: y  you want you
8 loss: 0.633387 Prediction: yf you want you
9 loss: 0.473833 Prediction: yf you want you
10 loss: 0.342214 Prediction: yf you want you
11 loss: 0.253288 Prediction: if you want you
12 loss: 0.193647 Prediction: if you want you
13 loss: 0.143812 Prediction: if you want you
14 loss: 0.105368 Prediction: if you want you
15 loss: 0.0752206 Prediction: if you want you
16 loss: 0.0532051 Prediction: if you want you
17 loss: 0.0381469 Prediction: if you want you
18 loss: 0.0275081 Prediction: if you want you
19 loss: 0.0199943 Prediction: if you want you
20 loss: 0.0148345 Prediction: if you want you
21 loss: 0.0113403 Prediction: if you want you

'\n0 loss: 2.35377 Prediction: uuuuuuuuuuuuuuu\n1 loss: 2.21383 Prediction: yy you y    you\n2 loss: 2.04317 Prediction: yy yoo       ou\n3 loss: 1.85869 Prediction: yy  ou      uou\n4 loss: 1.65096 Prediction: yy you  a   you\n5 loss: 1.40243 Prediction: yy you yan  you\n6 loss: 1.12986 Prediction: yy you wann you\n7 loss: 0.907699 Prediction: yy you want you\n8 loss: 0.687401 Prediction: yf you want you\n9 loss: 0.508868 Prediction: yf you want you\n10 loss: 0.379423 Prediction: yf you want you\n11 loss: 0.282956 Prediction: if you want you\n12 loss: 0.208561 Prediction: if you want you\n\n...\n\n'